In [126]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, from_json, col, explode,lower, collect_list,concat_ws
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [129]:
spark = init_spark()
movies_df = spark.read.csv(os.path.join("data/movies_metadata.csv"), header=True, inferSchema=True)

# Select columns we are interested in
movies_df = movies_df.select("id", "title", "genres").distinct()
genres_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True)
])

# Convert the "genres" column to an array of structs
movies_df = movies_df.withColumn("genres", from_json(col("genres"), ArrayType(genres_schema)))

movies_df = movies_df.select("id", "title", explode(col("genres")).alias("genre")) \
               .select("id", "title", col("genre.name").alias("genre_name")) \
               .dropna()

# Group the data by id and title, and collect the genre names into a list
movies_df = movies_df.groupBy("id", "title").agg(collect_list("genre_name").alias("genres"))

# Combine the genre names into one string
movies_df = movies_df.withColumn("genres", concat_ws(",", "genres"))
movies_df.show(truncate=False)


+------+--------------------------------------------------+------------------------------------------------+
|id    |title                                             |genres                                          |
+------+--------------------------------------------------+------------------------------------------------+
|10001 |Young Einstein                                    |Comedy,Science Fiction                          |
|100010|Flight Command                                    |Drama,War                                       |
|10002 |Mona Lisa                                         |Drama,Crime,Romance                             |
|100024|Bloodwork                                         |Horror,Thriller                                 |
|100032|Released                                          |Drama,Action                                    |
|10004 |Desperation                                       |Drama,Fantasy,Horror,Thriller,Mystery           |
|10006 |Wild Seven 

In [130]:
# Split the genres column into multiple columns
genres = movies_df.select("genres").rdd.flatMap(lambda x: x[0].split(',')).distinct().collect()
for genre in genres:
    movies_df = movies_df.withColumn(genre, col("genres").contains(genre).cast("int"))
movies_df.show(truncate=False)



+------+--------------------------------------------------+------------------------------------------------+---------------+-----+-------+------+--------+------+-------+------+-----------+-------+--------+------+---+-----+-------+---------+---------+-------+-----+-------+
|id    |title                                             |genres                                          |Science Fiction|Drama|Romance|Horror|Thriller|Action|Fantasy|Family|Documentary|Western|TV Movie|Comedy|War|Crime|Mystery|Adventure|Animation|History|Music|Foreign|
+------+--------------------------------------------------+------------------------------------------------+---------------+-----+-------+------+--------+------+-------+------+-----------+-------+--------+------+---+-----+-------+---------+---------+-------+-----+-------+
|10001 |Young Einstein                                    |Comedy,Science Fiction                          |1              |0    |0      |0     |0       |0     |0      |0     |0    

In [134]:
# Create a feature vector for each movie
assembler = VectorAssembler(inputCols=genres, outputCol="features")
movies_df = assembler.transform(movies_df)
movies_df.show()

+------+--------------------+--------------------+---------------+-----+-------+------+--------+------+-------+------+-----------+-------+--------+------+---+-----+-------+---------+---------+-------+-----+-------+--------------------+
|    id|               title|              genres|Science Fiction|Drama|Romance|Horror|Thriller|Action|Fantasy|Family|Documentary|Western|TV Movie|Comedy|War|Crime|Mystery|Adventure|Animation|History|Music|Foreign|            features|
+------+--------------------+--------------------+---------------+-----+-------+------+--------+------+-------+------+-----------+-------+--------+------+---+-----+-------+---------+---------+-------+-----+-------+--------------------+
| 10001|      Young Einstein|Comedy,Science Fi...|              1|    0|      0|     0|       0|     0|      0|     0|          0|      0|       0|     1|  0|    0|      0|        0|        0|      0|    0|      0|(20,[0,11],[1.0,1...|
|100010|      Flight Command|           Drama,War|      

In [135]:
from pyspark.ml.clustering import KMeans

# Train and fit to kmeans model (can change k and seed later)
kmeans = KMeans(k=20)
model = kmeans.fit(movies_df.select("features"))

In [137]:
def recommend_movies(moviesInput, ratings):
    # Create a dataframe from the input movies and ratings
    input_df = spark.createDataFrame(list(zip(moviesInput, ratings)), schema=["title", "rating"])

    # Join with the movies dataframe to get the feature vectors for the input movies
    input_movies = movies.join(input_df, "title", "right")
    input_movies = assembler.transform(input_movies)

    # Use the kmeans model to predict clusters for the input movies
    predictions = model.transform(input_movies)

    # Get the cluster label with the most input movies
    most_common_cluster = predictions.groupBy("prediction").count().orderBy(col("count").desc()).first()["prediction"]

    # Get the movies in the most common cluster
    cluster_movies = predictions.filter(col("prediction") == most_common_cluster).drop("features")

    # Remove the input movies from the cluster
    cluster_movies = cluster_movies.join(input_df, "title", "left_anti")

    # Select the top 10 recommended movies based on the highest predicted ratings
    recommended_movies = cluster_movies.orderBy(col("prediction"), col("prediction")).limit(10)

    # Return a list of recommended movie titles
    return recommended_movies.select("title").rdd.flatMap(lambda x: x).collect()

moviesInput = ["The Dark Knight", "Inception", "Interstellar"]
ratings = [5.0, 4.0, 3.5]
recommended = recommend_movies(moviesInput, ratings)
print(recommended)
)

IllegalArgumentException: Output column features already exists.

In [ ]:
# Read the user's list of rated movies
user_ratings = spark.createDataFrame([
    ("17015", 5),
    ("31174", 2),
    ("137", 7),
    ("129", 10)
], ["id", "rating"])

# Join the user_ratings DataFrame with the movies DataFrame
joined = movies.join(user_ratings, on=["id"], how="inner")

# Group by genre_name and compute the average rating
genre_ratings = joined.groupBy("genre_name") \
                      .agg({"rating": "avg"}) \
                      .withColumnRenamed("avg(rating)", "genre_rating")
# joined.show()
genre_ratings.show()


AnalysisException: Column 'genre_name' does not exist. Did you mean one of the following? [genres, Crime, Drama, Western, Adventure, Fantasy, Foreign, Romance, Action, Animation, Comedy, Documentary, Horror, Mystery, TV Movie, Thriller, War, features, rating, title, Family, History, Music, id, Science Fiction];
'Aggregate ['genre_name], ['genre_name, avg(rating#7743L) AS avg(rating)#7796]
+- Project [id#7163, title#7178, genres#7235, Science Fiction#7295, Drama#7300, Romance#7306, Horror#7313, Thriller#7321, Action#7330, Fantasy#7340, Family#7351, Documentary#7363, Western#7376, TV Movie#7390, Comedy#7405, War#7421, Crime#7438, Mystery#7456, Adventure#7475, Animation#7495, History#7516, Music#7538, Foreign#7561, features#7717, rating#7743L]
   +- Join Inner, (id#7163 = id#7742)
      :- Project [id#7163, title#7178, genres#7235, Science Fiction#7295, Drama#7300, Romance#7306, Horror#7313, Thriller#7321, Action#7330, Fantasy#7340, Family#7351, Documentary#7363, Western#7376, TV Movie#73